In [1]:
import re
import pandas as pd
import numpy as np
import camelot
import tabula
from fpdf import FPDF
from scripts.pdf_parser_class import PDFparser

In [2]:
PDF_EX_1 = "../../task_description/examples/GFS 5760519.pdf"
PDF_EX_2 = "../../task_description/examples/Sysco PO#_338-4243823.pdf"

## Наивный метод определения структуры таблицы
Наивный метод заключается в попытке преобразовать текст таблицы, полученный с помощью кастомного модуля [PDFparser](pdf_parser_test.ipynb), обратно в пдф для передачи в один из [специализированных модулей](compare_modules.ipynb). Расчет на то, что они смогут лучше распознать структуру вновь сгенерированной пдф-таблицы (без контекста).

In [3]:
def recreate_pdf(PDF, name, trimm=False):
    my_parser = PDFparser()
    df = my_parser.get_rows_marked(PDF)
    
    tbl_rows = (df
                .loc[df["mark"] == "tbl_row", 0]
                .reset_index(drop=True))
    
    # без "висячих" неполных строк
    if trimm:
        tbl_rows = tbl_rows.loc[range(0, len(tbl_rows), 2)]        
    
    pdf = FPDF()
    pdf.add_page(orientation="L")
    pdf.set_font("Courier", size=8)
    for k, st in enumerate(tbl_rows):
        pdf.cell(w=0,
                 h=5,
                 border=0,
                 align="L",
                 ln=1,
                 txt=st)
    pdf.output(name)
    return None

In [4]:
recreate_pdf(PDF_EX_1, "generated_df_ex_1.pdf", trimm=True)
recreate_pdf(PDF_EX_2, "generated_df_ex_2.pdf")

Наивный подход дает крайне неудовлетворительные результаты (фактически, те же, что и [при подаче на вход оригинального пдф](compare_modules.ipynb)):

In [5]:
df_from_ex_1_tab = tabula.read_pdf("generated_df_ex_1.pdf",
                                   stream=True, pages="all")
df_from_ex_1_tab.head()

,1,1 500G,00066958017026 SPICE SESAME SEED BLK,|,Unnamed: 4,125774996/04,Unnamed: 6,192,Unnamed: 8,7.0000,.1108,Unnamed: 11
0,2,12 212ML,01899500212117 PEPPERCORN GRN IN BRINE,|,NaN,128118027/07,NaN,27.0,NaN,45.00,0.5867,NaN
1,3,6 2L,00066958145422 PICKLE CORNICHON EXTRA FINE,|,NaN,130749612/03,NaN,36.0,NaN,59.64,3.1264,NaN
2,4,1 3.3KG,00066958150334 CHERRY AMARENA 18/20,|,NaN,133107710/04,NaN,20.0,NaN,46.50,0.7071,NaN
3,5,1 227G,00084909007971 PASTE TAMARIND,|,NaN,529081699/01,NaN,99.0,NaN,4.06,0.0573,NaN
4,6,1 280MG,00066958251376 POWDER XANTHAN GUM FOR TEXTURE,|,NaN,125323920/05,NaN,1.0,NaN,8.76,0.0554,NaN


In [6]:
df_from_ex_2_tab = tabula.read_pdf("generated_df_ex_2.pdf",
                                   stream=True, pages="all")
df_from_ex_2_tab.head()

,1,1.1,454 GMQUALFST,Unnamed: 3,181725,Unnamed: 5,0116871SPICE CARDAMOM GRND,Unnamed: 7,1.2,23.4700,.0000
0,2,1.0,1000GMDGF,NaN,152666,NaN,0216556GELATIN SHEET GOLD LEAF,NaN,2,62.17,0.0
1,3,1.0,2LROYALCM,NaN,152599,NaN,1438755SYRUP AGAVE NECTAR,NaN,6,41.11,0.0
2,4,1.0,200 GMROYALCM,NaN,152403,NaN,4303356FOOD ADDTV TRANSGLUTAMINA,NaN,2,56.47,0.0
3,5,12.0,500 MLVNITEAU,NaN,143030,NaN,4873257VINEGAR RED WINE AGED 7.1,NaN,1,55.80,0.0
4,6,1.0,454 GRROYAL,NaN,181728,NaN,5163090SPICE CARAWAY SEED WHL,NaN,1,8.23,0.0


In [7]:
camelot_results_ex_1 = camelot.read_pdf("generated_df_ex_1.pdf",
                                        flavor="stream",
                                        split_text=True,
                                        suppress_stdout=True,
                                        pages="all")
df_from_ex_1_cam = camelot_results_ex_1[0].df
df_from_ex_1_cam.head()

,0
0,1 1 500G 00066958017026 SPICE SESAME SEE...
1,2 12 212ML 01899500212117 PEPPERCORN GRN I...
2,3 6 2L 00066958145422 PICKLE CORNICHON...
3,4 1 3.3KG 00066958150334 CHERRY AMARENA 1...
4,5 1 227G 00084909007971 PASTE TAMARIND ...


In [8]:
camelot_results_ex_2 = camelot.read_pdf("generated_df_ex_2.pdf",
                                        flavor="stream",
                                        split_text=True,
                                        suppress_stdout=True,
                                        pages="all")
df_from_ex_2_cam = camelot_results_ex_2[0].df
df_from_ex_2_cam.head()

,0
0,1 1 454 GM QUALFST 181725 ...
1,2 1 1000GM DGF 152666 ...
2,3 1 2L ROYALCM 152599 ...
3,4 1 200 GM ROYALCM 152403 ...
4,5 12 500 ML VNITEAU 143030 ...


## Эвристический подход к определению структуры таблицы
1. Определяем колонки на основании промежутков между элементами в каждой строке. Пример строки:  

In [9]:
my_parser = PDFparser()
df = my_parser.get_rows_marked(PDF_EX_2)

row = (df
       .loc[df["mark"] == "tbl_row", 0]
       .reset_index(drop=True)[0])

print(row)

1   1      454 GM   QUALFST           181725          0116871   SPICE CARDAMOM GRND         1      23.4700                .0000


Вот та же строка с указанием колиества пробелов между ее элементами:

In [10]:
tokens = re.findall(r"\s+", row)

ws = []
for g, tk in enumerate(tokens):
    ws.append(len(tk))
ws.insert(0, 0)
ws.append(0)

row_splitted = re.split(r"\s+", row)

ROW_DEM = ""
for h, w in enumerate(row_splitted):
    ROW_DEM += " (" + str(ws[h]) + ") " + w
ROW_DEM += " (" + str(ws[-1]) + ") "

print(ROW_DEM)

 (0) 1 (3) 1 (6) 454 (1) GM (3) QUALFST (11) 181725 (10) 0116871 (3) SPICE (1) CARDAMOM (1) GRND (9) 1 (6) 23.4700 (16) .0000 (0) 


Отсюда выводим первую [эвристику](https://github.com/woldemarg/pdf_parser#%D1%8D%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8-%D1%82%D1%80%D0%B5%D0%B1%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F-%D0%BA-%D0%B2%D1%85%D0%BE%D0%B4%D0%BD%D1%8B%D0%BC-%D1%84%D0%B0%D0%B9%D0%BB%D0%B0%D0%BC) для распознания колонок в тексте: строки таблицы разбиваются на колонки на основаниии промежутков между элементами. Элементы объединяются в колонку до тех пор, пока расстояние между ними не превышает 1 пробела, а растояние до ближайшей группы элементов справа и слева составляет более 1 пробела. "Правильное" количество колонок при этом определяется по моде с учетом всех строк в таблице. Если количество колонок в отдельной строке не равно моде, значения в ячейках заменяются на н/д.  

In [11]:
def split_to_cols(row_org):
    tkns = re.findall(r"\s+", row_org)

    whsp = [] # num of spaces between words in a row
    for i, t in enumerate(tkns):
        whsp.append(len(t))
    whsp.insert(0, 0)
    whsp.append(0)

    row_spl = re.split(r"\s+", row_org)

    cols = []
    cols.append(row_spl[0])

    i = 1
    while i < len(row_spl[:-1]):
        el = row_spl[i]
        if whsp[i + 1] != 1:
            i += 1
        else:
            j = 1
            while whsp[i + j] == 1:
                el += " " + row_spl[i + j]
                j += 1
            i = i + j
        cols.append(el.strip())
    cols.append(row_spl[-1])

    return cols

In [12]:
# одна строка
print(split_to_cols(row))

['1', '1', '454 GM', 'QUALFST', '181725', '0116871', 'SPICE CARDAMOM GRND', '1', '23.4700', '.0000']


In [13]:
# таблица
tbl_rows = (df
           .loc[df["mark"] == "tbl_row", 0]
           .reset_index(drop=True))

splitted_rows = []
for rw in tbl_rows:    
    splitted_rows.append(split_to_cols(rw))

rows_len = list(map(len, splitted_rows))
len_mode = max(rows_len, key=rows_len.count)

for i, rw in enumerate(splitted_rows):
    if len(rw) != len_mode:
        splitted_rows[i] = [np.nan] * len_mode

init_df = pd.DataFrame(splitted_rows)

init_df

,0,1,2,3,4,5,6,7,8,9
0,1,1,454 GM,QUALFST,181725,0116871,SPICE CARDAMOM GRND,1,23.4700,.0000
1,2,1,1000GM,DGF,152666,0216556,GELATIN SHEET GOLD LEAF,2,62.1700,.0000
2,3,1,2L,ROYALCM,152599,1438755,SYRUP AGAVE NECTAR,6,41.1100,.0000
3,4,1,200 GM,ROYALCM,152403,4303356,FOOD ADDTV TRANSGLUTAMINA,2,56.4700,.0000
4,5,12,500 ML,VNITEAU,143030,4873257,VINEGAR RED WINE AGED 7.1,1,55.8000,.0000
5,6,1,454 GR,ROYAL,181728,5163090,SPICE CARAWAY SEED WHL,1,8.2300,.0000
6,7,1,100 GM,SULPIZO,050536,7103403,TRUFFLE BLACK PASTE,1,51.7000,.0000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Если для парсинга пдф использовать метод *parse_method="camelot"* (!работает на порядок дольше - см. [ноутбук](compare_modules.ipynb)), то, по крайней мере, для некоторых пдф результат заметно лучше.

In [14]:
# та же таблица, но полученная с помощью
# метода parse_metho="camelot"
cam_parser = PDFparser(parse_method="camelot")
df = cam_parser.get_rows_marked(PDF_EX_2)

tbl_rows = (df
           .loc[df["mark"] == "tbl_row", 0]
           .reset_index(drop=True))

splitted_rows = []
for rw in tbl_rows:    
    splitted_rows.append(split_to_cols(rw))

rows_len = list(map(len, splitted_rows))
len_mode = max(rows_len, key=rows_len.count)

for i, rw in enumerate(splitted_rows):
    if len(rw) != len_mode:
        splitted_rows[i] = [np.nan] * len_mode

init_df = pd.DataFrame(splitted_rows)

init_df

,0,1,2,3,4,5,6,7,8,9
0,1,1,454 GM,QUALFST,181725,0116871,SPICE CARDAMOM GRND,1,23.4700,.0000
1,2,1,1000GM,DGF,152666,0216556,GELATIN SHEET GOLD LEAF,2,62.1700,.0000
2,3,1,2L,ROYALCM,152599,1438755,SYRUP AGAVE NECTAR,6,41.1100,.0000
3,4,1,200 GM,ROYALCM,152403,4303356,FOOD ADDTV TRANSGLUTAMINA,2,56.4700,.0000
4,5,12,500 ML,VNITEAU,143030,4873257,VINEGAR RED WINE AGED 7.1,1,55.8000,.0000
5,6,1,454 GR,ROYAL,181728,5163090,SPICE CARAWAY SEED WHL,1,8.2300,.0000
6,7,1,100 GM,SULPIZO,050536,7103403,TRUFFLE BLACK PASTE,1,51.7000,.0000
7,8,1,227 GM,NMFOODS,184112,8637498,SPICE CHILES CHIPOTLE SMK,1,11.1100,.0000


2. Как видим, в некоторых случаях в одну колонку попали значения явно из разных столбцов, на разделенные недостаточным количеством пробелов. Для решения проблемы вводим еще одну [эвристику](https://github.com/woldemarg/pdf_parser#%D1%8D%D0%B2%D1%80%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B8-%D1%82%D1%80%D0%B5%D0%B1%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D1%8F-%D0%BA-%D0%B2%D1%85%D0%BE%D0%B4%D0%BD%D1%8B%D0%BC-%D1%84%D0%B0%D0%B9%D0%BB%D0%B0%D0%BC): если в одну колонку попадают элементы, состоящие исключительно из цифр, и внутри этой колонки можно разделить текст каждой из строк по "пробелам" таким образом, что "цифровые" элементы формируют отдельную субколонку во всех строках, то исходная колонка разбивается на соответствующее количесво новых колонок.

In [15]:
new_parser = PDFparser(parse_method="camelot")
new_df = new_parser.get_rows_marked(PDF_EX_1)

tbl_rows_ex_1 = (new_df
                .loc[new_df["mark"] == "tbl_row", 0]
                .reset_index(drop=True))
    
# без "висячих" неполных строк
tbl_rows_ex_1 = tbl_rows_ex_1.loc[range(0, len(tbl_rows_ex_1), 2)]

splitted_rows = []
for rw in tbl_rows_ex_1:    
    splitted_rows.append(split_to_cols(rw))

rows_len = list(map(len, splitted_rows))
len_mode = max(rows_len, key=rows_len.count)

for i, rw in enumerate(splitted_rows):
    if len(rw) != len_mode:
        splitted_rows[i] = [np.nan] * len_mode

init_df = pd.DataFrame(splitted_rows)

# для примера по одной колонке
# сначала разбиваем текст по-элементно
multi_col = (init_df[2]
             .str.split(expand=True)
             .fillna(""))
multi_col

,0,1,2,3,4,5
0,00066958017026,SPICE,SESAME,SEED,BLK,
1,01899500212117,PEPPERCORN,GRN,IN,BRINE,
2,00066958145422,PICKLE,CORNICHON,EXTRA,FINE,
3,00066958150334,CHERRY,AMARENA,18/20,,
4,00084909007971,PASTE,TAMARIND,,,
5,00066958251376,POWDER,XANTHAN,GUM,FOR,TEXTURE
6,00066958800116,DREDGE,CHICK,BROWN'S,CUST,
7,00066958120122,WATER,ORANGE,BLOSSOM,,
8,00847972000139,SALT,SEA,SMKD,,
9,40044738102382,PASTE,CURRY,YEL,,


In [16]:
def readjust_cols(original_col):
    splitted_cols = (original_col.str.split(expand=True)
                     .fillna(""))

    if splitted_cols.shape[1] == 1:
        return pd.DataFrame(original_col)

    is_digit = (splitted_cols
                .apply(lambda col:
                       all(str(elem).isdigit() for elem in col),
                       axis=0))

    rsnd_series = []

    idx = 0
    while idx < len(is_digit):
        cur_col = splitted_cols[idx]
        if is_digit[idx]:
            idx += 1
        else:
            j = 1
            while idx + j < len(is_digit):
                if ~is_digit[idx + j]:
                    cur_col += " " + splitted_cols[idx + j]
                    j += 1
                else:
                    break
            idx += j
        rsnd_series.append(cur_col.str.strip())
    rsnd_cols = pd.DataFrame(rsnd_series).transpose()

    return rsnd_cols

In [17]:
readjust_cols(init_df[2])

,0,1
0,00066958017026,SPICE SESAME SEED BLK
1,01899500212117,PEPPERCORN GRN IN BRINE
2,00066958145422,PICKLE CORNICHON EXTRA FINE
3,00066958150334,CHERRY AMARENA 18/20
4,00084909007971,PASTE TAMARIND
5,00066958251376,POWDER XANTHAN GUM FOR TEXTURE
6,00066958800116,DREDGE CHICK BROWN'S CUST
7,00066958120122,WATER ORANGE BLOSSOM
8,00847972000139,SALT SEA SMKD
9,40044738102382,PASTE CURRY YEL


In [18]:
# еще один пример, того, как работает євристика п.2
example = [["Aa A1 1 aaa a-2 011 a-d zz"],
           ["Bb 2 2 bbb b-2 009 b-c zz"],
           ["Cc 3 3 ccc c-2 007 c-d zz"]]

exp= pd.DataFrame(example)
exp

,0
0,Aa A1 1 aaa a-2 011 a-d zz
1,Bb 2 2 bbb b-2 009 b-c zz
2,Cc 3 3 ccc c-2 007 c-d zz


In [19]:
readjust_cols(exp[0])

,0,2,3,5,6
0,Aa A1,1,aaa a-2,011,a-d zz
1,Bb 2,2,bbb b-2,009,b-c zz
2,Cc 3,3,ccc c-2,007,c-d zz


3.Собираем все правила в один метод и тестируем на сложной таблице (!пока без обработки заголовков и "висячих" строк):
![сложная таблица](gfs_5760519_page_1.png)

In [20]:
def get_table(tbl_rows):
    splitted_rows = []
    for rw in tbl_rows:
        rw = rw.replace("|", "")
        splitted_rows.append(split_to_cols(rw))

    rows_len = list(map(len, splitted_rows))
    len_mode = max(rows_len, key=rows_len.count)

    for i, rw in enumerate(splitted_rows):
        if len(rw) != len_mode:
            splitted_rows[i] = [np.nan] * len_mode

    init_df = pd.DataFrame(splitted_rows)

    i, offset = 0, 0
    while i < init_df.shape[1]:
        new_cols = readjust_cols(init_df[i])
        init_df = pd.concat([init_df.iloc[:, :i],
                             new_cols,
                             init_df.iloc[:, i + 1:]],
                            axis=1)
        init_df.columns = range(init_df.shape[1])
        offset = new_cols.shape[1] - 1
        i += 1 + offset

    return init_df

In [21]:
df_from_pdf = get_table(tbl_rows_ex_1)
df_from_pdf

,0,1,2,3,4,5,6,7,8,9
0,1,1,500G,00066958017026,SPICE SESAME SEED BLK,1257749,96/04,192,7.0000,.1108
1,2,12,212ML,01899500212117,PEPPERCORN GRN IN BRINE,1281180,27/07,27,45.0000,.5867
2,3,6,2L,00066958145422,PICKLE CORNICHON EXTRA FINE,1307496,12/03,36,59.6400,3.1264
3,4,1,3.3KG,00066958150334,CHERRY AMARENA 18/20,1331077,10/04,20,46.5000,.7071
4,5,1,227G,00084909007971,PASTE TAMARIND,5290816,99/01,99,4.0600,.0573
5,6,1,280MG,00066958251376,POWDER XANTHAN GUM FOR TEXTURE,1253239,20/05,1,8.7600,.0554
6,7,1,5KG,00066958800116,DREDGE CHICK BROWN'S CUST,1354729,10/02,10,21.9500,.9651
7,8,1,125ML,00066958120122,WATER ORANGE BLOSSOM,1323173,99/01,3,3.4700,.0344
8,9,6,125G,00847972000139,SALT SEA SMKD,1344865,20/04,4,42.0000,.1739
9,10,24,400G,40044738102382,PASTE CURRY YEL,9854956,30/01,2,78.4800,2.1021
